# Reading List of HTML Documents from CSV File

This section contains a modified example based on the [reading documents page](http://chemdataextractor.org/docs/reading) of the Chem Data Extractor (CDE) documentation. 

A csv file containing the details of 10 documents is included (articles_list.csv). the file contains the location of a pdf copy stored locally as well as a link to the web version of the same article.

The functions used in the **Read CSV List.ipynb** and the **cde_read_html.py** examples are combined in this example to allow reading from the list of html documents. The main change is the function which retrieves the html document, saves a temporary copy and returns the contents for processing.

In [1]:
# The line of code (LOC) below imports the document object from the CDE library 
from chemdataextractor import Document

# import library for managing files
from pathlib import Path
import sys

# import library for managing csv files
import csv

# library for handling html requests (reading online pages)
import requests

# A function for getting a list of files from the directory
# This will be modified to get the list from a csv file
def get_files_list (source_dir):
    i_counter = 0
    files_list = []
    for filepath in sorted(source_dir.glob('*.pdf')):
        i_counter += 1
        files_list.append(filepath)
    return files_list

# A function for getting a list of unique occurrecnces 
# returns an array of element names and their occurrence count
def get_uniques(cde_doc):
    uniques={}
    for chement in cde_doc.cems:
        if not chement.text in uniques:
            uniques[chement.text] = 1
        else:
            uniques[chement.text] += 1
    return uniques

# A function for getting the entity with most occurrecnces  
# retuns two values: the entity name and the count
def get_max(uniques):
    max_val = 0
    max_lbl = ""
    for chement in uniques:
        if uniques[chement] > max_val:
            max_val = uniques[chement]
            max_lbl = chement.replace('\n',' ')
    return max_lbl, max_val

# get the data from the csv_file, assuming first column is integer id
def get_csv_data(input_file, id_field):
    csv_data = {}
    fieldnames=[]
    with open(input_file, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if fieldnames==[]:
                fieldnames=list(row.keys())
            csv_data[int(row[id_field])]=row
    return csv_data, fieldnames

# writes data to the given file name
def write_csv_data(values, filename):
    fieldnames = []
    for item in values.keys():
        for key in values[item].keys():
            if not key in fieldnames:
                fieldnames.append(key)
    #write back to a new csv file
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for key in values.keys():
            writer.writerow(values[key])

# a function tha teads a html document from the web,
# saves a copy and returns a file object in read mode
def get_html_file(article_url = "https://pubs.rsc.org/ko/content/articlehtml/2019/cp/c8cp05975f"):
    # set request header
    req_head = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'}
    # get the page content 
    html_response = requests.get(article_url, headers = req_head)
    #save the content as a temporary file in the local disk
    f= open("temp.html","w+")
    f.write(str(html_response.content))
    f.close()

    # open de temporary file and read in binary mode
    return open("temp.html", 'rb')
            
# A function which read a list of files from directory
# and performs a basic analysis of the documents looking
# for the most mentioned entity
def cde_read_pdfs(pdf_path = "./pdfs"):
    pdf_dir= Path(pdf_path)
    files_list = get_files_list(pdf_dir)
    print(files_list)
    for a_file in files_list:
        file_name = a_file.name
        pdf_f = open(a_file, 'rb')
        doc = Document.from_file(pdf_f)
        uniques = get_uniques(doc)
        max_lbl, max_val = get_max(uniques)       
        print(file_name, "Unique entities:", len(uniques), "Most common entity:", max_lbl, max_val)

# A function which read a list of files from a csv file
# and performs a basic analysis of the documents looking
# for the most mentioned entity
# modified version of the one which reads from directory
def cde_read_pdfs_csv(csv_name = "./articles_list.csv"):
    articles_list, column_names = get_csv_data(csv_name, "id")
    for a_file in articles_list:
        file_name = articles_list[a_file]['filename']
        file_title = articles_list[a_file]['title']
        pdf_f = open(file_name, 'rb')
        doc = Document.from_file(pdf_f)
        uniques = get_uniques(doc)
        max_lbl, max_val = get_max(uniques)       
        print(file_title, "Unique entities:", len(uniques), "Most common entity:", max_lbl, max_val)

# A function which read a list of urls from a csv file
# and performs a basic analysis of the retrieved documents
# looking for the most mentioned entity
# modified version of the one which reads from directory
def cde_read_html_csv(csv_name = "./articles_list.csv"):
    articles_list, column_names = get_csv_data(csv_name, "id")
    for a_file in articles_list:
        file_url = articles_list[a_file]['url']        
        file_title = articles_list[a_file]['title']
        pdf_f = get_html_file(file_url)
        doc = Document.from_file(pdf_f)
        uniques = get_uniques(doc)
        max_lbl, max_val = get_max(uniques)       
        print(file_title, "Unique entities:", len(uniques), "Most common entity:", max_lbl, max_val)

After the functions are declared, we can directly call the cde_read_html_csv function and see its results. 

In [2]:
#read url list from csv file
cde_read_html_csv("./articles_list.csv")

Investigation of ZSM-5 catalysts for dimethylether conversion using inelastic neutron scattering Unique entities: 12 Most common entity: ZSM-5 3
Elementary Steps in the Formation of Hydrocarbons from Surface Methoxy Groups in HZSMâ€‘5 Seen by Synchrotron Infrared Microspectroscopy Unique entities: 37 Most common entity: methanol 3
Machine Learning for Catalysis Informatics: Recent Applications and Prospects Unique entities: 18 Most common entity: CO2 2
Hydrogen adsorption on transition metal carbides: a DFT study Unique entities: 49 Most common entity: hydrogen 56
QM/MM simulations identify the determinants of catalytic activity differences between type II dehydroquinase enzymes Unique entities: 115 Most common entity: Arg 35
Supporting Information ReOx/TiO2 â€“ a recyclable solid catalyst for deoxydehydration Unique entities: 376 Most common entity: MediaBox[0 0 612 792] 11
ReOx/TiO2: A Recyclable Solid Catalyst for Deoxydehydration Unique entities: 98 Most common entity: Rhenium 8
Ef

## Comparing the performance of the three versions

Up to this point we have three versions which do the same operation.
The results vary a little between each, becuse some online versions only include the abstract, not the full article and also because the supporting information file (sixth article in the list) is a link to the pdf version.

However, the advantage of the last example seems to be that the html parsing version appear to be faster than the pdf versions. 
The following cells compare them by running them and timestamping beginning and end.

In [3]:
#import library for handling date-time
from datetime import datetime
# get start time
start = datetime.now()
# read from directory
cde_read_pdfs("./pdfs")
# get end time
end = datetime.now()
# print start-end time
print("Results of reading from directory")
print("Start:", start)
print("End:", end)
print("Processing", end-start)

[WindowsPath('pdfs/1-s2.0-S0926860X18305003-main.pdf'), WindowsPath('pdfs/acscatal.9b01820.pdf'), WindowsPath('pdfs/acscatal.9b04186.pdf'), WindowsPath('pdfs/c8cp05975f.pdf'), WindowsPath('pdfs/c8ob00066b.pdf'), WindowsPath('pdfs/cs5b01936_si_Proof.pdf'), WindowsPath('pdfs/cs5b01936MainProof.pdf'), WindowsPath('pdfs/Decarolis2018_Article_EffectOfParticleSizeAndSupport.pdf'), WindowsPath('pdfs/fchem-07-00182.pdf'), WindowsPath('pdfs/PhysRevB.66.224405.pdf')]
1-s2.0-S0926860X18305003-main.pdf Unique entities: 108 Most common entity: DME 29
acscatal.9b01820.pdf Unique entities: 148 Most common entity: methanol 40
acscatal.9b04186.pdf Unique entities: 279 Most common entity: H 25
c8cp05975f.pdf Unique entities: 56 Most common entity: hydrogen 54
c8ob00066b.pdf Unique entities: 123 Most common entity: Arg 37
cs5b01936_si_Proof.pdf Unique entities: 79 Most common entity: TiO2 27
cs5b01936MainProof.pdf Unique entities: 88 Most common entity: ReOx 30
Decarolis2018_Article_EffectOfParticleSizeA

In [4]:
start = datetime.now()
# read pdf list from csv file 
cde_read_pdfs_csv("./articles_list.csv")
# get end time
end = datetime.now()
# print start-end time
print("Results of reading pdf list from csv file")
print("Start:", start)
print("End:", end)
print("Processing", end-start)

Investigation of ZSM-5 catalysts for dimethylether conversion using inelastic neutron scattering Unique entities: 108 Most common entity: DME 29
Elementary Steps in the Formation of Hydrocarbons from Surface Methoxy Groups in HZSMâ€‘5 Seen by Synchrotron Infrared Microspectroscopy Unique entities: 148 Most common entity: methanol 40
Machine Learning for Catalysis Informatics: Recent Applications and Prospects Unique entities: 279 Most common entity: H 25
Hydrogen adsorption on transition metal carbides: a DFT study Unique entities: 56 Most common entity: hydrogen 54
QM/MM simulations identify the determinants of catalytic activity differences between type II dehydroquinase enzymes Unique entities: 123 Most common entity: Arg 37
Supporting Information ReOx/TiO2 â€“ a recyclable solid catalyst for deoxydehydration Unique entities: 79 Most common entity: TiO2 27
ReOx/TiO2: A Recyclable Solid Catalyst for Deoxydehydration Unique entities: 88 Most common entity: ReOx 30
Effect of Particle S

In [5]:
start = datetime.now()
# read url list from csv file 
cde_read_html_csv("./articles_list.csv")
# get end time
end = datetime.now()
# print start-end time
print("Results of reading pdf list from csv file")
print("Start:", start)
print("End:", end)
print("Processing", end-start)

Investigation of ZSM-5 catalysts for dimethylether conversion using inelastic neutron scattering Unique entities: 12 Most common entity: ZSM-5 3
Elementary Steps in the Formation of Hydrocarbons from Surface Methoxy Groups in HZSMâ€‘5 Seen by Synchrotron Infrared Microspectroscopy Unique entities: 37 Most common entity: methanol 3
Machine Learning for Catalysis Informatics: Recent Applications and Prospects Unique entities: 18 Most common entity: CO2 2
Hydrogen adsorption on transition metal carbides: a DFT study Unique entities: 49 Most common entity: hydrogen 56
QM/MM simulations identify the determinants of catalytic activity differences between type II dehydroquinase enzymes Unique entities: 115 Most common entity: Arg 35
Supporting Information ReOx/TiO2 â€“ a recyclable solid catalyst for deoxydehydration Unique entities: 376 Most common entity: MediaBox[0 0 612 792] 11
ReOx/TiO2: A Recyclable Solid Catalyst for Deoxydehydration Unique entities: 98 Most common entity: Rhenium 8
Ef